# SETUP

## Import Libraries

In [209]:
from pymongo import MongoClient
import pandas as pd

## Read MongoDB

In [210]:
client = MongoClient()
db = client["test"]

# MATCHAPPS

## Matchapps Initalization

### Create Matchapps DataFrame

In [211]:
matchapps_collection = db["matchapps"]
matchapps_df = matchapps_collection.find()

### Flatten, reorganize, and clean Data

In [212]:
matchapps_df = pd.json_normalize(matchapps_df)

matchapps_column_order = [
    "_id",
    "metadata.matchNumber",
    "metadata.scouterName",
    "metadata.robotTeam",
    "metadata.robotPosition",
    "leftStartingZone",
    "climb",
    "trapNotes",
    "autoNotes.near",
    "autoNotes.mid",
    "autoNotes.far",
    "autoNotes.amp",
    "autoNotes.miss",
    "teleNotes.near",
    "teleNotes.mid",
    "teleNotes.far",
    "teleNotes.amp",
    "teleNotes.miss"   
]

matchapps_df = matchapps_df[matchapps_column_order]
matchapps_df.sort_values("metadata.matchNumber", inplace=True)
matchapps_df

,_id,metadata.matchNumber,metadata.scouterName,metadata.robotTeam,metadata.robotPosition,leftStartingZone,climb,trapNotes,autoNotes.near,autoNotes.mid,autoNotes.far,autoNotes.amp,autoNotes.miss,teleNotes.near,teleNotes.mid,teleNotes.far,teleNotes.amp,teleNotes.miss
0,65e21a8090db904a4ca5e888,1,Jim,1496,red_1,False,park,0,8,2,5,3,3,2,7,3,2,8
1,65e21a8090db904a4ca5e88a,1,Jim,874,red_2,True,failed,0,9,4,1,5,7,8,2,1,9,9
2,65e21a8090db904a4ca5e88c,1,Jim,701,red_3,True,none,1,5,8,2,8,1,8,6,9,2,8
3,65e21a8090db904a4ca5e88e,1,Jim,8204,blue_1,False,failed,0,5,6,4,3,5,5,8,5,7,9
4,65e21a8090db904a4ca5e890,1,Jim,6921,blue_2,False,source,0,9,5,6,8,9,7,2,4,2,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2390,65e21a8c90db904a4ca5fb34,399,Jim,9413,red_3,True,center,1,9,9,7,1,6,4,6,3,4,8
2392,65e21a8c90db904a4ca5fb38,399,Jim,9351,blue_2,True,amp,0,1,9,4,4,3,8,1,4,8,4
2388,65e21a8c90db904a4ca5fb30,399,Jim,7842,red_1,True,source,1,5,4,5,6,2,3,5,5,7,4
2389,65e21a8c90db904a4ca5fb32,399,Jim,331,red_2,True,failed,0,3,1,0,2,3,0,1,9,7,0


### Initialize Output DF

In [213]:
matchapps_output_df = pd.DataFrame()

## Matchapps Function and Variable Initializations

### Check Alliance Harmonization Function Definition

In [214]:
def check_team_harmonization_in_alliance(t1, t2, t3):
    t1_status, t2_status, t3_status = False, False, False
    if t1 not in ["failed", "none", "park"]:
        if t1 == t2:
            t1_status = True
            t2_status = True
        if t1 == t3:
            t1_status = True
            t3_status = True
    elif t2 not in ["failed", "none", "park"]:
        if t2 == t3:
            t2_status = True
            t3_status = True
    return t1_status, t2_status, t3_status

### Complete List of all Teams in DF

In [215]:
team_list = matchapps_df["metadata.robotTeam"].unique().tolist()

### Complete Range of All Matches

In [216]:
match_range = matchapps_df["metadata.matchNumber"].max()

## Matchapps Data Analysis

### Auto Data Analysis Calculations

#### Auto Speaker Notes Average (near, mid, far)

In [217]:
matchapps_output_df["Auto Speaker Notes Near Avg"] = matchapps_df.groupby("metadata.robotTeam")["autoNotes.near"].mean()
matchapps_output_df["Auto Speaker Notes Mid Avg"] = matchapps_df.groupby("metadata.robotTeam")["autoNotes.mid"].mean()
matchapps_output_df["Auto Speaker Notes Far Avg"] = matchapps_df.groupby("metadata.robotTeam")["autoNotes.far"].mean()

#### Auto Amp Notes Average

In [218]:
matchapps_output_df["Auto Notes Amp Avg"] = matchapps_df.groupby("metadata.robotTeam")["autoNotes.amp"].mean()

#### Auto Percent of Miss Notes

In [219]:
matchapps_output_df["Auto Percent of Missed Notes"] = None
for team in team_list:
    missed_notes = matchapps_df[matchapps_df["metadata.robotTeam"] == team]["autoNotes.miss"].sum()
    total_notes = matchapps_df[matchapps_df["metadata.robotTeam"] == team]["autoNotes.near"].sum()
    + matchapps_df[matchapps_df["metadata.robotTeam"] == team]["autoNotes.mid"].sum()
    + matchapps_df[matchapps_df["metadata.robotTeam"] == team]["autoNotes.far"].sum()
    + matchapps_df[matchapps_df["metadata.robotTeam"] == team]["autoNotes.amp"].sum()
    + missed_notes
    matchapps_output_df.at[team, "Auto Percent of Missed Notes"] = missed_notes / total_notes

#### Auto All Notes Average

In [220]:
matchapps_output_df["Auto Notes Avg"] = (matchapps_output_df["Auto Notes Amp Avg"] +
                               matchapps_output_df["Auto Speaker Notes Near Avg"] +
                               matchapps_output_df["Auto Speaker Notes Mid Avg"] +
                               matchapps_output_df["Auto Speaker Notes Far Avg"]) / 4

### Teleop Data Analysis Calculations

#### Teleop Speaker Notes Average (near, mid, far)

In [221]:
matchapps_output_df["Tele Speaker Notes Near Avg"] = matchapps_df.groupby("metadata.robotTeam")["teleNotes.near"].mean()
matchapps_output_df["Tele Speaker Notes Mid Avg"] = matchapps_df.groupby("metadata.robotTeam")["teleNotes.mid"].mean()
matchapps_output_df["Tele Speaker Notes Far Avg"] = matchapps_df.groupby("metadata.robotTeam")["teleNotes.far"].mean()

#### Teleop Amp Notes Average

In [222]:
matchapps_output_df["Tele Notes Amp Avg"] = matchapps_df.groupby("metadata.robotTeam")["teleNotes.amp"].mean()

#### Teleop All Notes Average

In [223]:
matchapps_output_df["Tele Notes Avg"] = (matchapps_output_df["Tele Notes Amp Avg"] +
                               matchapps_output_df["Tele Speaker Notes Near Avg"] +
                               matchapps_output_df["Tele Speaker Notes Mid Avg"] +
                               matchapps_output_df["Tele Speaker Notes Far Avg"]) / 4

#### Teleop Percent of Missed Notes

In [224]:
matchapps_output_df["Tele Percent of Missed Notes"] = None
for team in team_list:
    missed_notes = matchapps_df[matchapps_df["metadata.robotTeam"] == team]["teleNotes.miss"].sum()
    total_notes = matchapps_df[matchapps_df["metadata.robotTeam"] == team]["teleNotes.near"].sum()
    + matchapps_df[matchapps_df["metadata.robotTeam"] == team]["teleNotes.mid"].sum()
    + matchapps_df[matchapps_df["metadata.robotTeam"] == team]["teleNotes.far"].sum()
    + matchapps_df[matchapps_df["metadata.robotTeam"] == team]["teleNotes.amp"].sum()
    + missed_notes
    matchapps_output_df.at[team, "Percent of Missed Notes"] = missed_notes / total_notes

### Other Data Analysis Calculations

#### Trap Notes

In [225]:
matchapps_output_df["Trap Notes Avg"] = matchapps_df.groupby("metadata.robotTeam")["trapNotes"].mean()

#### Percent of Successful Climbs and Successful Climbs Total

In [226]:
matchapps_output_df["Percent of Successful Climbs"] = None
matchapps_output_df["Successful Climbs Total"] = None

for team in team_list:
    climb_list = matchapps_df.loc[matchapps_df["metadata.robotTeam"] == team, "climb"].tolist()
    total_climbs = 0
    successful_climbs = 0
    for climb_type in climb_list:
        if climb_type == "failed":
            total_climbs += 1
        elif climb_type == "amp" or climb_type == "source" or climb_type == "center":
            total_climbs += 1
            successful_climbs += 1
    if total_climbs == 0:
        matchapps_output_df.at[team, "Percent of Successful Climbs"] = 0
        
    else:
        matchapps_output_df.at[team, "Percent of Successful Climbs"] = successful_climbs / total_climbs
    
    matchapps_output_df.at[team, "Successful Climbs Total"] = successful_climbs

#### Harmonized Climbs Total

In [227]:
matchapps_output_df["Harmonized Climbs Total"] = 0

for match in range(1, match_range + 1):
    match_number_df = matchapps_df['metadata.matchNumber'] == match
    robots = matchapps_df[match_number_df]
    robots.sort_values("metadata.robotPosition", inplace=True)
    
    teams_harmonized = []
    
    for i in range(6):
        teams_harmonized.append([robots.iloc[i]["metadata.robotTeam"], robots.iloc[i]["climb"], False])

    teams_harmonized[0][2], teams_harmonized[1][2], teams_harmonized[2][2] = check_team_harmonization_in_alliance(teams_harmonized[0][1], teams_harmonized[1][1], teams_harmonized[2][1])
    teams_harmonized[3][2], teams_harmonized[4][2], teams_harmonized[5][2] = check_team_harmonization_in_alliance(teams_harmonized[3][1], teams_harmonized[4][1], teams_harmonized[5][1])

    for checked_team in teams_harmonized:
        if checked_team[2]:
            matchapps_output_df.at[checked_team[0], "Harmonized Climbs Total"] += 1

C:\Users\benja\AppData\Local\Temp\ipykernel_22124\3676841220.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  robots.sort_values("metadata.robotPosition", inplace=True)


#### Percent of Harmonized Climbs out of Successful Climbs

In [228]:
matchapps_output_df["Percent of Harmonized Climbs out of Successful Climbs"] = 0

for team in team_list:
    matchapps_output_df.at[team, "Percent of Harmonized Climbs out of Successful Climbs"] = matchapps_output_df.at[team, "Harmonized Climbs Total"] / matchapps_output_df.at[team, "Successful Climbs Total"]

C:\Users\benja\AppData\Local\Temp\ipykernel_22124\948994976.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.1935483870967742' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  matchapps_output_df.at[team, "Percent of Harmonized Climbs out of Successful Climbs"] = matchapps_output_df.at[team, "Harmonized Climbs Total"] / matchapps_output_df.at[team, "Successful Climbs Total"]


#### Percent of Times Robot Left Starting Zone During Autos

In [229]:
matchapps_output_df["Percent of Times Robot Left Starting Zone During Autos"] = matchapps_df.groupby("metadata.robotTeam")["leftStartingZone"].mean()

### Custom Metric Data Analysis Calculations

#### Endgame Custom Metric

In [230]:
matchapps_output_df["Endgame Custom Metric"] = ((matchapps_output_df["Percent of Successful Climbs"] * 5) ** 2 + (matchapps_output_df["Percent of Harmonized Climbs out of Successful Climbs"] * 5) ** 2 + (matchapps_output_df["Trap Notes Avg"] * 5) ** 2) / 3
matchapps_output_df["Endgame Custom Metric"] = matchapps_output_df["Endgame Custom Metric"].apply(lambda x: round(x, 1))

## Matchapps Finalize and Send Data

### Send Data

In [231]:
matchapps_output_df.to_json("../server/static/analyzed_data.json", orient='records', default_handler=str)

# SUPERAPPS

## Supperapps Initalization

### Create Superapps DataFrame

In [232]:
superapps_collection = db["superapps"]
superapps_df = superapps_collection.find()

### Flatten, reorganize, and clean Data

In [233]:
superapps_df = pd.json_normalize(superapps_df)
superapps_df.sort_values("metadata.matchNumber", inplace=True)
superapps_df

,_id,defense,defended,__v,metadata.scouterName,metadata.matchNumber,metadata.robotTeam,metadata.robotPosition,fouls.inBot,fouls.damageBot,fouls.tipEntangBot,fouls.pinBot,fouls.podiumFoul,fouls.zoneFoul,fouls.stageFoul,fouls.overExtChute,humanShooter.highNotes.amp,humanShooter.highNotes.source,humanShooter.highNotes.center
0,65e21a8c90db904a4ca5fb3c,someDef,False,0,Jim,1,2652,red_1,9,8,9,6,4,9,2,8,True,True,True
1,65e21a8c90db904a4ca5fb3e,someDef,False,0,Jim,1,331,red_2,9,7,8,8,1,8,3,3,False,False,True
2,65e21a8c90db904a4ca5fb40,noDef,True,0,Jim,1,2652,red_3,9,2,3,6,3,7,6,8,False,False,True
3,65e21a8c90db904a4ca5fb42,noDef,True,0,Jim,1,701,blue_1,1,1,5,4,5,3,7,8,False,True,True
4,65e21a8c90db904a4ca5fb44,fullDef,False,0,Jim,1,6824,blue_2,6,2,8,8,6,3,0,7,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2390,65e21a9790db904a4ca60de8,fullDef,True,0,Jim,399,6465,red_3,7,7,6,3,3,0,2,9,False,True,True
2392,65e21a9790db904a4ca60dec,fullDef,True,0,Jim,399,3285,blue_2,3,7,9,8,8,6,4,2,False,True,False
2388,65e21a9790db904a4ca60de4,fullDef,True,0,Jim,399,2215,red_1,5,9,5,6,8,1,4,1,True,False,False
2389,65e21a9790db904a4ca60de6,someDef,True,0,Jim,399,2034,red_2,2,7,8,2,5,2,5,9,False,True,False


### Initialize Output DF

In [234]:
superapps_output_df = pd.DataFrame()

## Superapps Data Analysis

### Fouls

In [235]:
superapps_output_df["Foul - In Bot"] = superapps_df.groupby("metadata.robotTeam")["fouls.inBot"].mean()
superapps_output_df["Foul - Damage Bot"] = superapps_df.groupby("metadata.robotTeam")["fouls.damageBot"].mean()
superapps_output_df["Foul - Tip Entangle Bot"] = superapps_df.groupby("metadata.robotTeam")["fouls.tipEntangBot"].mean()
superapps_output_df["Foul - Pin Bot"] = superapps_df.groupby("metadata.robotTeam")["fouls.pinBot"].mean()
superapps_output_df["Foul - Podium"] = superapps_df.groupby("metadata.robotTeam")["fouls.podiumFoul"].mean()
superapps_output_df["Foul - Zone"] = superapps_df.groupby("metadata.robotTeam")["fouls.zoneFoul"].mean()
superapps_output_df["Foul - Stage"] = superapps_df.groupby("metadata.robotTeam")["fouls.stageFoul"].mean()
superapps_output_df["Foul - Over Exc Chute"] = superapps_df.groupby("metadata.robotTeam")["fouls.overExtChute"].mean()

superapps_output_df["Average Fouls per Match"] = None

for team in team_list:
    superapps_output_df.at[team, "Average Fouls per Match"] = superapps_output_df.at[team, "Foul - In Bot"] + superapps_output_df.at[team, "Foul - Damage Bot"] + superapps_output_df.at[team, "Foul - Tip Entangle Bot"] + superapps_output_df.at[team, "Foul - Pin Bot"] + superapps_output_df.at[team, "Foul - Podium"] + superapps_output_df.at[team, "Foul - Zone"] + superapps_output_df.at[team, "Foul - Stage"] + superapps_output_df.at[team, "Foul - Over Exc Chute"]

### Defense

In [236]:
superapps_output_df["Full Defense Total"] = superapps_df[superapps_df["defense"] == "fullDef"].groupby("metadata.robotTeam").size()
superapps_output_df["Some Defense Total"] = superapps_df[superapps_df["defense"] == "someDef"].groupby("metadata.robotTeam").size()
superapps_output_df["No Defense Total"] = superapps_df[superapps_df["defense"] == "noDef"].groupby("metadata.robotTeam").size()

## Superapps Finalize and Send Data

### Send Data

In [237]:
superapps_output_df.to_json("../server/static/analyzed_data.json", orient='records', default_handler=str)